In [1]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import Dense, Dropout, Flatten,BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import TensorBoard,EarlyStopping,ReduceLROnPlateau
from tensorflow.keras import optimizers
from tensorflow.keras.utils import to_categorical
import numpy as np
# argument
num_classes = 7
batch_size = 256
epochs = 1000

2023-12-26 06:32:21.196225: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1


In [2]:
# Load data
with open("../data/challenges-in-representation-learning-facial-expression-recognition-challenge/fer2013/fer2013.csv") as f:
    content = f.readlines()

lines = np.array(content)
num_of_instances = lines.size


# Phan chia train va test set
x_train, y_train, x_public_test, y_public_test,x_private_test, y_private_test = [], [], [], [],[], []

# ------------------------------
# transfer train and test set data
for i in range(1, num_of_instances):
    try:
        emotion, img, usage = lines[i].split(",")
        val = img.split(" ")
        pixels = np.array(val, 'float32')
        emotion = to_categorical(emotion, num_classes)

        if 'Training' in usage:
            y_train.append(emotion)
            x_train.append(pixels)
        if 'PublicTest' in usage:
            y_public_test.append(emotion)
            x_public_test.append(pixels)
        if 'PrivateTest' in usage:
            y_private_test.append(emotion)
            x_private_test.append(pixels)
    except:
        print("", end="")

# Chuyen thanh numpy array
x_train = np.array(x_train, 'float32')
y_train = np.array(y_train, 'float32')
x_public_test = np.array(x_public_test, 'float32')
y_public_test = np.array(y_public_test, 'float32')
x_private_test = np.array(x_private_test, 'float32')
y_private_test = np.array(y_private_test, 'float32')
x_train /= 255  # normalize inputs between [0, 1]
x_public_test /= 255
x_private_test /= 255

# Chuyen thanh tensor
x_train = x_train.reshape(x_train.shape[0], 48, 48, 1)
x_train = x_train.astype('float32')
x_public_test = x_public_test.reshape(x_public_test.shape[0], 48, 48, 1)
x_public_test = x_public_test.astype('float32')
x_private_test = x_private_test.reshape(x_private_test.shape[0], 48, 48, 1)
x_private_test = x_private_test.astype('float32')

# Tao batch generator
# gen = ImageDataGenerator(
#     rescale=1/255,
#     featurewise_center=True,
#     featurewise_std_normalization=True,
#     rotation_range=20,
#     width_shift_range=0.2,
#     height_shift_range=0.2,
#     horizontal_flip=True,
#     validation_split=0.2
# )
gen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    validation_split=0.2
)
train_generator = gen.flow(x_train, y_train, batch_size=batch_size)

In [3]:
# Directory where the TensorBoard logs will be stored
log_dir = './logs'
# Setting up TensorBoard callback
tb_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

2023-12-26 06:32:29.308363: I tensorflow/core/profiler/lib/profiler_session.cc:164] Profiler session started.
2023-12-26 06:32:29.308819: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcuda.so.1
2023-12-26 06:32:29.337204: I tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1391] Profiler found 2 GPUs
2023-12-26 06:32:29.337328: W tensorflow/stream_executor/platform/default/dso_loader.cc:59] Could not load dynamic library 'libcupti.so.10.1'; dlerror: libcupti.so.10.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-10.2/lib64:/usr/local/cuda-10.2/lib64
2023-12-26 06:32:29.337682: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcupti.so
2023-12-26 06:32:29.338573: I tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1513] CUPTI activity buffer flushed


# Create model

In [4]:
import sys
sys.path.append('../.')
model = Sequential()
model.add(Conv2D(64, (5, 5), activation='relu',padding="same", input_shape=(48, 48, 1)))
model.add(MaxPooling2D(pool_size=(5, 5), strides=(2, 2)))
model.add(Dropout(0.2))
model.add(BatchNormalization())
model.add(Conv2D(64, (3, 3),padding="same", activation='relu'))
model.add(Conv2D(64, (3, 3),padding="same", activation='relu'))
model.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2)))
model.add(Dropout(0.2))
model.add(BatchNormalization())
model.add(Conv2D(128, (3, 3),padding="same", activation='relu'))
model.add(Conv2D(128, (3, 3),padding="same", activation='relu'))
model.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2)))
model.add(Dropout(0.2))
model.add(BatchNormalization())

model.add(Flatten())
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.2))
model.add(BatchNormalization())
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.2))
model.add(BatchNormalization())
model.add(Dense(num_classes, activation='softmax'))

2023-12-26 06:32:29.349722: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:982] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-12-26 06:32:29.350118: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1716] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: GeForce RTX 2080 computeCapability: 7.5
coreClock: 1.71GHz coreCount: 46 deviceMemorySize: 7.76GiB deviceMemoryBandwidth: 417.23GiB/s
2023-12-26 06:32:29.350157: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:982] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-12-26 06:32:29.350456: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1716] Found device 1 with properties: 
pciBusID: 0000:02:00.0 name: GeForce RTX 2080 computeCapability: 7.5
coreClock: 1.8GHz coreCount: 46 deviceMemorySize: 7.79GiB deviceMemoryBandwidth: 417.23GiB/s

In [5]:
# Define early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=50)

# Define learning rate scheduler
lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=10)

model.compile(loss='categorical_crossentropy'
                , optimizer=optimizers.Adam()
                , metrics=['accuracy']
              )

# Train model
model.fit(train_generator,
           batch_size=batch_size,
           epochs=epochs,
           validation_data=(x_public_test, y_public_test),
           callbacks=[tb_callback, early_stopping, lr_scheduler]) 

#overall evaluation
public_score = model.evaluate(x_public_test, y_public_test)
print(f"Public Test loss:'{public_score[0]},Public Test accuracy:', {100*public_score[1]}")
private_score = model.evaluate(x_private_test, y_private_test)
print(f"Private Test loss:'{private_score[0]},Private Test accuracy:', {100*private_score[1]}")


/home/danieldu/anaconda3/envs/kaggle/lib/python3.8/site-packages/keras_preprocessing/image/image_data_generator.py:720: UserWarning: This ImageDataGenerator specifies `featurewise_center`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '
/home/danieldu/anaconda3/envs/kaggle/lib/python3.8/site-packages/keras_preprocessing/image/image_data_generator.py:728: UserWarning: This ImageDataGenerator specifies `featurewise_std_normalization`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '


Epoch 1/1000


2023-12-26 06:32:31.055618: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcublas.so.10
2023-12-26 06:32:31.224890: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudnn.so.7


  1/113 [..............................] - ETA: 0s - loss: 2.6653 - accuracy: 0.0977WARNING:tensorflow:From /home/danieldu/anaconda3/envs/kaggle/lib/python3.8/site-packages/tensorflow/python/ops/summary_ops_v2.py:1277: stop (from tensorflow.python.eager.profiler) is deprecated and will be removed after 2020-07-01.
Instructions for updating:
use `tf.profiler.experimental.stop` instead.
  4/113 [>.............................] - ETA: 4s - loss: 2.6615 - accuracy: 0.1367

2023-12-26 06:32:32.329156: I tensorflow/core/profiler/lib/profiler_session.cc:164] Profiler session started.
2023-12-26 06:32:32.357425: I tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1513] CUPTI activity buffer flushed
2023-12-26 06:32:32.358816: I tensorflow/core/profiler/internal/gpu/device_tracer.cc:223]  GpuTracer has collected 315 callback api events and 315 activity events. 
2023-12-26 06:32:32.368664: I tensorflow/core/profiler/rpc/client/save_profile.cc:176] Creating directory: ./logs/train/plugins/profile/2023_12_26_06_32_32
2023-12-26 06:32:32.372984: I tensorflow/core/profiler/rpc/client/save_profile.cc:182] Dumped gzipped tool data for trace.json.gz to ./logs/train/plugins/profile/2023_12_26_06_32_32/danieldu.trace.json.gz
2023-12-26 06:32:32.382125: I tensorflow/core/profiler/rpc/client/save_profile.cc:176] Creating directory: ./logs/train/plugins/profile/2023_12_26_06_32_32
2023-12-26 06:32:32.383624: I tensorflow/core/profiler/rpc/client/save_profile.cc:182] D

113/113 [==============================] - 6s 52ms/step - loss: 2.3201 - accuracy: 0.1855 - val_loss: 7.9539 - val_accuracy: 0.1159
Epoch 2/1000
113/113 [==============================] - 6s 50ms/step - loss: 1.9051 - accuracy: 0.2213 - val_loss: 52.2736 - val_accuracy: 0.1694
Epoch 3/1000
113/113 [==============================] - 6s 50ms/step - loss: 1.8466 - accuracy: 0.2330 - val_loss: 34.1711 - val_accuracy: 0.2115
Epoch 4/1000
113/113 [==============================] - 6s 50ms/step - loss: 1.8266 - accuracy: 0.2439 - val_loss: 81.5526 - val_accuracy: 0.2026
Epoch 5/1000
113/113 [==============================] - 6s 50ms/step - loss: 1.8159 - accuracy: 0.2465 - val_loss: 170.9063 - val_accuracy: 0.1296
Epoch 6/1000
113/113 [==============================] - 6s 50ms/step - loss: 1.8134 - accuracy: 0.2479 - val_loss: 298.2223 - val_accuracy: 0.1298
Epoch 7/1000
113/113 [==============================] - 6s 50ms/step - loss: 1.8045 - accuracy: 0.2532 - val_loss: 392.4608 - val_accura

In [6]:
model.save("../output/facial.h5")

In [7]:
outputs = np.argmax(model.predict(x_private_test), axis=-1)
csv_file='../submission.csv'
np.savetxt(csv_file, outputs, delimiter=",", fmt='%s', header='', comments='')